In [6]:
import importlib

import numpy as np
import pandas as pd

import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

import scipy
from scipy.stats import pearsonr, spearmanr

import pyBigWig
import pysam

from Modules import utils, plot_utils

Useful global variables

In [7]:
importlib.reload(utils)
importlib.reload(plot_utils)
prop_cycle = plt.rcParams["axes.prop_cycle"]
colors = prop_cycle.by_key()["color"]
# plt.rcParams.update({'axes.labelsize': 16})
with open("/home/alex/shared_folder/Human/T2T-CHM13v2.0_NCBI2UCSC.txt") as f:
    T2T_to_ucsc = dict(line.split() for line in f.readlines())

Compute quantiles ans sums

In [8]:
bwfiles = {
    # "Ctrl": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C44_trimmed_run2_paired_T2T.bw",
    # "IP1": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C43_trimmed_run2_paired_T2T.bw",
    # "IP2": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/HN00205099_RawFASTQ_RPE1_WTH3K9me3_run2_paired_T2T.bw",
    "Ctrl_pp1": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C44_trimmed_run2_paired_T2T_properonce.bw",
    "IP1_pp1": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C43_trimmed_run2_paired_T2T_properonce.bw",
    "IP2_pp1": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/HN00205099_RawFASTQ_RPE1_WTH3K9me3_run2_paired_T2T_properonce.bw",
    # "Ctrl_pp1_mm3": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C44_trimmed_run2_paired_T2T_properonce_minmapq3.bw",
    # "IP1_pp1_mm3": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C43_trimmed_run2_paired_T2T_properonce_minmapq3.bw",
    # "IP2_pp1_mm3": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/HN00205099_RawFASTQ_RPE1_WTH3K9me3_run2_paired_T2T_properonce_minmapq3.bw",
}

In [5]:
# Compute quantiles and sums
q_list = np.array([0.9, 0.95, 0.99, 0.999])
quantiles = {}
sums = {}
clipped_sums = {}
for file_id, bwfile in bwfiles.items():
    signals = utils.load_bw(bwfile, nantonum=False)
    full = np.concatenate(list(signals.values()))
    quantiles[bwfile] = np.quantile(full, q_list)
    sums[bwfile] = sum(np.sum(v) for v in signals.values())
    q99 = quantiles[bwfile][np.where(q_list == 0.99)]
    clipped_sums[bwfile] = sum(
        np.sum(np.clip(v, a_min=None, a_max=q99)) for v in signals.values()
    )
# Save quantiles
quantiles = pd.DataFrame(quantiles, index=[f"quantile_{q}" for q in q_list]).T
quant_filename = utils.safe_filename(
    "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/quantiles_run2_properonce.csv"
)
quantiles.to_csv(quant_filename)
print(quantiles)
# Save sums
df_sums = pd.DataFrame(sums, index=["Total_counts"], columns=sums.keys()).T
df_sums["Clip_q0.99_total_counts"] = clipped_sums.values()
df_sums = df_sums.astype(int)
sums_filename = utils.safe_filename(
    "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/sums_run2_properonce.csv"
)
df_sums.to_csv(sums_filename)
df_sums

/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/quantiles_run2.csv exists, changing filename to /home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/quantiles_run2(1).csv
                                                    quantile_0.9  \
/home/alex/shared_folder/Judith-H3K9me3/results...           6.0   
/home/alex/shared_folder/Judith-H3K9me3/results...           8.0   
/home/alex/shared_folder/Judith-H3K9me3/results...          58.0   
/home/alex/shared_folder/Judith-H3K9me3/results...           6.0   
/home/alex/shared_folder/Judith-H3K9me3/results...           7.0   
/home/alex/shared_folder/Judith-H3K9me3/results...          56.0   

                                                    quantile_0.95  \
/home/alex/shared_folder/Judith-H3K9me3/results...            7.0   
/home/alex/shared_folder/Judith-H3K9me3/results...            9.0   
/home/alex/shared_folder/Judith-H3K9me3/results...           66.0   
/home/alex/shared_folder/Judith

,Total_counts,Clip_q0.99_total_counts
/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C44_trimmed_run2_paired_T2T_properonce.bw,10083182935,9978968696
/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C43_trimmed_run2_paired_T2T_properonce.bw,12909345130,12765793469
/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/HN00205099_RawFASTQ_RPE1_WTH3K9me3_run2_paired_T2T_properonce.bw,115395390776,114225681744
/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C44_trimmed_run2_paired_T2T_properonce_minmapq3.bw,9419627723,9361102674
/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C43_trimmed_run2_paired_T2T_properonce_minmapq3.bw,11750988524,11667731722
/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/HN00205099_RawFASTQ_RPE1_WTH3K9me3_run2_paired_T2T_properonce_minmapq3.bw,106382352924,105909394476


In [4]:
# Compute quantiles
q_list = [0.9, 0.95, 0.99, 0.999]
quantiles = {}
for file_id, bwfile in bwfiles.items():
    signals = utils.load_bw(bwfile, nantonum=False)
    full = np.concatenate(list(signals.values()))
    quantiles[bwfile] = np.quantile(full, q_list)
quantiles = pd.DataFrame(quantiles, index=[f"quantile_{q}" for q in q_list]).T
filename = utils.safe_filename(
    "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/quantiles_run2.csv"
)
quantiles.to_csv(filename)
quantiles

In [6]:
# Compute sum, and thresholded to 99e centile sum
sums = {}
clipped_sums = {}
for file_id, bwfile in bwfiles.items():
    filesum = 0
    clipped_filesum = 0
    with pyBigWig.open(bwfile) as bw:
        for chr_id in bw.chroms():
            values = bw.values(chr_id, 0, -1, numpy=True)
            filesum += np.sum(values)
            np.clip(
                values,
                a_min=None,
                a_max=quantiles.loc[bwfile, "quantile_0.99"],
                out=values,
            )
            clipped_filesum += np.sum(values)
    sums[bwfile] = filesum
    clipped_sums[bwfile] = clipped_filesum
df = pd.DataFrame(sums, index=["Total_counts"], columns=sums.keys()).T
df["Clip_q0.99_total_counts"] = clipped_sums.values()
df = df.astype(int)
filename = utils.safe_filename(
    "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/sums_run2.csv"
)
df.to_csv(filename)
df

,Total_counts,Clip_q0.99_total_counts
/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C44_trimmed_run2_paired_T2T.bw,20967397902,20801592514
/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C43_trimmed_run2_paired_T2T.bw,26920413596,26608732992
/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/HN00205099_RawFASTQ_RPE1_WTH3K9me3_run2_paired_T2T.bw,249342779072,246931352992
